In [340]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torchsummary import summary
import sentencepiece as spm
import numpy as np
import pandas as pd
import torchtext
import nltk
from konlpy.tag import Okt
from torchtext.data import Field, BucketIterator, TabularDataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from collections import defaultdict
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
import pickle
import regex as re
from gensim.models import FastText
from konlpy.tag import *
import xgboost as xgb
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score

In [6]:
xgb_clf = xgb.XGBClassifier()

print(xgb_clf)

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=None, reg_lambda=None,
              scale_pos_weight=None, subsample=None, tree_method=None,
              validate_parameters=None, verbosity=None)


In [228]:
ft_model = FastText.load('0806_fasttext_model_all_reviews_bigram_skipgram_epoch=5_WS=5_alpha=0.05_mincount=150_maxn=4')

In [9]:
mecab = Mecab()

In [10]:
okt = Okt()

In [19]:
tag_list = ['Noun', 'Verb', 'Adjective', 'Adverb']
dataset = '200428_밀착력_with_rating'

In [126]:
data = pd.read_excel('{}.xlsx'.format(dataset), index_col=0)

In [127]:
data = data.dropna()

In [128]:
data = data.sample(len(data))

In [129]:
data['리뷰'] = [' '.join([token for token, pos in okt.pos(x) if pos in tag_list]) for x in tqdm(data['리뷰'])]

100%|██████████| 11115/11115 [00:43<00:00, 255.55it/s]


In [299]:
data['리뷰']

index
11084         용량 식구 많은 집 좋을듯 향 보습 보통 거품 좋음 깨끗히 씻어져 개운 함강 추
1829     발림 성도 좋고 색도 진해서 좋아요 자르고 난 후 딱 입 감기는게 좋은데 각질 부각...
1965     갯잇 뷰티 위 제품 해서 구매 해봤는데 발림 탁 현상 다 좋은데 저 넘 번 들거 린...
8696                 색상 자연스럽고 좋았음 붉은기 커버 잘 안되는데 촉촉하게 표현 되요
2512                                  이전 시즌 느낌 다소 붉은 느낌 드네
                               ...                        
800           톤업 효과 좀 있고 발림 성도 괜찮은 편 저 좀 맞나 봐요 얼굴 계속 뭐 나네요
5510     늘 쓰던거에요 가볍고 밀착 잘되지 커버 약해서 몇번 더 레이어 링 해서 발라 화사해...
3129                     커버 괜찮은 것 같아요 지속 이만 하면 만족 미디엄 딱이네요
9180     겨땀 파크 아니지만 날 더워지니 점차 축축해져 데오드란트 필요한 시기 옴 원래 니베...
1304                             커버 잘 되긴 하는데 끈 적임 있어서 써지네요
Name: 리뷰, Length: 11115, dtype: object

In [300]:
corpus = data['리뷰']

In [324]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

X = np.asarray([[token for token in x.split(' ')] for x in data['리뷰'].values])

X = pad_sequences(X, maxlen=100, padding='post', truncating='post')

In [329]:
y = data['구분'].values

In [330]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=777)

In [347]:
hr_pred = xgb_clf.fit(X_train, y_train).predict(X_test)

In [348]:
print(classification_report(y_test, hr_pred))

              precision    recall  f1-score   support

           0       0.75      0.79      0.77      1803
           1       0.60      0.65      0.62      1131
           2       0.17      0.01      0.02        85
           3       0.54      0.34      0.42       316

    accuracy                           0.68      3335
   macro avg       0.51      0.45      0.46      3335
weighted avg       0.66      0.68      0.67      3335

